In [19]:
import os
import glob as glob
import pandas as pd
import numpy as np
from events_prep import *

In [20]:
data_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep/'
out_dir = '/home/finc/Dropbox/Projects/SelfReg/data/behavioral/prep_order/'

files = np.sort(os.listdir(data_dir))

for i in range(len(files)):
    sub = files[i][0:4]
    file_path = f'{data_dir}{sub}_surveyMedley_events.tsv'
    if not os.path.exists(file_path):
        print(f'Survey data not exist for sub-{sub}')
    else:
        file_name = os.path.basename(file_path)
        df = pd.read_csv(file_path, sep = '\t')
        df['trial_type'] = df['item_text'].map(get_items_order()) # remaping item_text to the unique item order
        df['trial_no'] = np.arange(1,41) # adding column storing original order
        df['subject'] = f'sub-{sub}' # adding column with subject ID in BIDS format 
        df = df.drop(['text', 'worker_id'], axis=1)  # dropping unncecessary columns
        df.to_csv(f'{out_dir}/sub-{sub}_surveyMedley_events.csv', index = False)
        
df.head()        

,movement_onset,onset,response_time,duration,coded_response,experiment_exp_id,item_coding,item_responses,item_text,possible_responses,stim_duration,survey,time_elapsed,junk,trial_type,trial_no,subject
0,10.017,1.007,3.722324,3.982,4.0,survey_medley,reverse,"66,89,71,82,77",I have difficulty maintaining my focus on proj...,"66,89,71,82,77",8500.0,grit,12319,False,Q06,1,sub-s650
1,16.890,12.322,-0.719676,3.982,2.0,survey_medley,reverse,"66,89,71,82,77",New ideas and projects sometimes distract me f...,"66,89,71,82,77",8500.0,grit,21934,False,Q01,2,sub-s650
2,26.506,21.937,-0.718676,3.982,3.0,survey_medley,reverse,"66,89,71,82,77","I do certain things that are bad for me, if th...","66,89,71,82,77",8500.0,brief,31141,False,Q13,3,sub-s650
3,36.665,31.143,0.234324,3.982,3.0,survey_medley,forward,"66,89,71,82,77",I am good at resisting temptation.,"66,89,71,82,77",8500.0,brief,40687,False,Q09,4,sub-s650
4,47.597,40.690,1.619324,3.982,4.0,survey_medley,reverse,"66,89,71,82,77",I often act without thinking through all the a...,"66,89,71,82,77",8500.0,brief,50166,False,Q21,5,sub-s650
